# Python for Data Analysis : Modeling Part

In [1]:
import pandas as pd
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00373/drug_consumption.data", header = None)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
feature_col_names = ['AGE', 'GENDER', 'EDUCATION_LEVEL', 'COUNTRY', 'ETHNICITY', 'NSCORE_VALUE', 
    'ESCORE_VALUE', 'OSCORE_VALUE', 'ASCORE_VALUE', 'CSCORE_VALUE', 'IMPULSIVENESS', 'SENSATION_SEEING']

predicted_class_names = ['CAFFEINE_CONSUMPTION']

X = df[feature_col_names].values
y = df[predicted_class_names].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
from sklearn import metrics
from sklearn.svm import SVC

In [ ]:
# Create the model
svm = SVC(kernel="rbf", C=2,random_state=0)

# Fit the model
svm.fit(X_train, y_train.ravel())

# Predict values
pred = svm.predict(X_test)

# Calculate the accuracy
accuracy = metrics.accuracy_score(y_test,pred)
print(f"The accuracy score is : {accuracy}")

In [ ]:
svm = SVC(kernel="linear", C=2,random_state=0)
svm.fit(X_train, y_train.ravel())
pred = svm.predict(X_test)
accuracy = metrics.accuracy_score(y_test,pred)
print(f"The accuracy score is : {accuracy}")

In [ ]:
import pickle
pickle.dump(svm, open('models/svm_prediction.pickle', 'wb'))

In [ ]:
svm = SVC(kernel="poly", C=1,random_state=50)
svm.fit(X_train, y_train.ravel())
pred = svm.predict(X_test)
accuracy = metrics.accuracy_score(y_test,pred)
print(f"The accuracy score is : {accuracy}")

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dtc = DecisionTreeClassifier(criterion='gini',max_depth=4,random_state=0)
clf_dtc.fit(X_train, y_train.ravel())
pred = clf_dtc.predict(X_test)
accuracy = metrics.accuracy_score(y_test,pred)
print(f"The accuracy score is : {accuracy}")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test,pred)
print(f"The accuracy score is : {accuracy}")